In [1]:
import dimod
import neal
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite, DWaveCliqueSampler

# Solve Natural Linear Program as QUBO with default Solver Advantage_system1.1 


In [2]:
A_int = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,5,2,3,1,1],
              [0,0,1,2,2,0]])
m, n = A_int.shape
vartype = dimod.Vartype.BINARY
c_int =  np.array([-1,2,-2,2,1,1])
b = np.array([6,8,5,2])
print(A_int.dot(np.array([2,0,0,0,1,2])))

[6 8 5 2]


# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in N^n$ with:

$\min c^T x $ with constraint $Ax=b$ 



This is equivalent to solving QUBO:

$min_x c^Tx + x^T A^T A x - 2 b^TAx + b^Tb$

eventually we need to set weights on the penalty parts $x^T A^T A x - 2 b^TAx + b^Tb$

In [3]:
def get_binary_form(c,bits):
    c_bin = []
    for c_j in c:
        c_bin += [c_j * 2**i for i in range(0,bits)]
    return np.array(c_bin)

In [4]:
# represent each x_i as 2 bits to the power of 2
bits = 2
# Set Penalty for Constraints aprox solution
P = max(abs(c_int))*4
c = get_binary_form(c_int,bits)
print(c)
A = np.array([get_binary_form(A_int[i], bits) for i in range(0,m)])
print(A)

[-1 -2  2  4 -2 -4  2  4  1  2  1  2]
[[ 0  0  1  2  2  4  3  6  4  8  1  2]
 [ 2  4  1  2  4  8  1  2  0  0  2  4]
 [ 1  2  5 10  2  4  3  6  1  2  1  2]
 [ 0  0  0  0  1  2  2  4  2  4  0  0]]


In [5]:
# Set up linear and quadratic coefficients
L = c.T - P * 2 * np.matmul(b.T,A)
Q = P * np.matmul(A.T,A)
offset = P * b.T.dot(b) -1
bqm = dimod.as_bqm(L, Q, offset, vartype)

In [6]:
exactSolver = dimod.ExactSolver()
sim = neal.SimulatedAnnealingSampler()
num_reads = 2500
exactSet = exactSolver.sample(bqm)
simSet = sim.sample(bqm, num_reads = num_reads)

In [7]:
def construct_x(sample, bits):
    x=[]
    for k in range(0,len(sample.keys()),bits):
        x += [sum(sample[k+j]* 2**j for j in range(0,bits))]
    return np.array(x)

In [8]:
def testSamples(sampleset, bits):
    print("Minimum Energy Level found in:",sampleset.first)
    X = []
    for sample, energy,  in sampleset.data(fields=['sample', 'energy']):
        x = construct_x(sample,bits)
        if all(A_int.dot(x)[i] == b[i] for i in range(0,m)) :
            X += [(x,energy)]
    return X

In [9]:
solutions_exact = testSamples(exactSet, bits)
for x, energy in solutions_exact :
    print("Found solution {} with Energy Level {}".format(x,energy))
        

Minimum Energy Level found in: Sample(sample={0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1}, energy=0.0, num_occurrences=1)
Found solution [2 0 0 0 1 2] with Energy Level 0.0


In [10]:
solutions_neal = testSamples(simSet, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_neal}
solutions_count = {x:0 for x in solutions_unique.keys()}
for x,energy in solutions_neal:
    solutions_count[str(x)]+=1
for x in solutions_unique.keys() :
    print("Found solution {} {} times with Energy Level {}".format(x,solutions_count[x],solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1}, energy=0.0, num_occurrences=1)
Found solution [2 0 0 0 1 2] 646 times with Energy Level 0.0


In [11]:
i=0
for sample, energy in exactSet.data(['sample','energy']):
    print(sample,energy)
    i+=1
    if i==5: break

{0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1} 0.0
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0} 4.0
{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0} 6.0
{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 1} 10.0
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0} 14.0


So we have an energy gap of 4 between solution (ground state) and no solution (first excited state)

# Find Embedding automatically by EmbeddingComposite (minorminer.find_embedding)

In [12]:
samplerComposite = EmbeddingComposite(DWaveSampler())
sampleset = samplerComposite.sample(bqm,num_reads = num_reads) # b2529319-cbc7-461c-a9c6-e4d2576b4a25

In [13]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,chain_break_fraction,energy,num_occurrences
0,0,1,0,0,0,0,0,0,1,0,0,1,0.000000,0.0,36
1,0,1,0,0,1,0,0,0,1,0,0,0,0.000000,4.0,57
2,1,1,0,0,0,0,0,0,1,0,1,0,0.000000,6.0,45
3,1,0,0,0,0,0,0,0,1,0,1,1,0.000000,10.0,81
4,1,0,0,0,1,0,0,0,1,0,1,0,0.000000,14.0,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,0,0,0,0,1,0,0,0,1,1,0,1,0.000000,1034.0,1
244,1,1,0,0,0,0,0,0,1,0,1,1,0.083333,168.0,1
245,1,0,0,0,1,1,1,0,0,0,0,0,0.083333,730.0,1
246,0,1,0,1,1,0,0,0,1,0,1,0,0.083333,1177.0,1


In [14]:
solutions_real = testSamples(sampleset, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_real}
for x in solutions_unique.keys() :
    print("Found solution {} with Energy Level {}".format(x,solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1}, energy=0.0, num_occurrences=36, chain_break_fraction=0.0)
Found solution [2 0 0 0 1 2] with Energy Level 0.0


# Find Embedding automatically by DWaveCliqueSampler 

(wraps minorminer.busclique.find_clique_embedding)

In [15]:
samplerClique = DWaveCliqueSampler()
sampleset = samplerClique.sample(bqm,num_reads = num_reads) # 9263c4dc-9701-462a-bfad-d990ca722c1f

In [16]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,chain_break_fraction,energy,num_occurrences
0,0,1,0,0,0,0,0,0,1,0,0,1,0.000000,0.0,97
1,0,1,0,0,1,0,0,0,1,0,0,0,0.000000,4.0,14
2,1,1,0,0,0,0,0,0,1,0,1,0,0.000000,6.0,2
3,1,0,0,0,0,0,0,0,1,0,1,1,0.000000,10.0,6
4,0,0,0,0,0,0,0,0,1,0,1,1,0.083333,51.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,0,1,1,1,0,0,0,0,0,0,0,0,0.000000,1267.0,1
264,1,1,1,1,0,0,0,0,1,0,0,0,0.083333,1587.0,1
265,0,1,0,1,1,0,1,0,1,0,0,0,0.000000,1698.0,1
266,0,1,0,1,0,0,0,1,0,0,1,0,0.000000,1710.0,1


In [17]:
solutions_real = testSamples(sampleset, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_real}
for x in solutions_unique.keys() :
    print("Found solution {} with Energy Level {}".format(x,solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1}, energy=0.0, num_occurrences=97, chain_break_fraction=0.0)
Found solution [2 0 0 0 1 2] with Energy Level 0.0


### So we found the solution more often by using clique sampler